In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

%matplotlib inline

## Dataset

In [2]:
%%capture
fashion_dataset = keras.datasets.cifar10

(train_images, train_labels), (test_images, test_labels) = fashion_dataset.load_data()

In [3]:
print(train_images.shape)
print(train_labels.shape)

(50000, 32, 32, 3)
(50000, 1)


In [4]:
print("BEFORE NORMALIZING:")
print("IMAGES: ",train_images.max()) # RANGE: 0-255
print("LABELS: ",train_labels.max()) # RANGE: 0-9

BEFORE NORMALIZING:
IMAGES:  255
LABELS:  9


In [5]:
train_images = train_images / 255.0
test_images = test_images / 255.0

print("AFTER NORMALIZING:")
print("IMAGES: ",train_images.max()) # RANGE: 0-1
print("IMAGES: ",test_images.max()) # RANGE: 0-1

AFTER NORMALIZING:
IMAGES:  1.0
IMAGES:  1.0


In [6]:
# plt.figure(figsize=(2,2))
# plt.imshow(train_images[1])

## Image Augmentation

In [15]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# EXPLORE MORE
'''
tf.image.resize      -   resize
image.img_to_array   -   IMG to NP_ARRAY
ImageDataGenerator   -   create augmented dataset
'''

## Network & Train

In [33]:
# NETWORK
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax'),
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # from_logits = False (DEFAULT) ## EXPLORE ##
              metrics=['accuracy'])

In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

In [ ]:
model.fit(train_images, train_labels, epochs=10, batch_size=64)

Epoch 1/10
782/782 [==============================] - 56s 71ms/step - loss: 2.1070 - accuracy: 0.3443
Epoch 2/10
782/782 [==============================] - 58s 75ms/step - loss: 1.9720 - accuracy: 0.4869
Epoch 3/10
782/782 [==============================] - 52s 66ms/step - loss: 1.9230 - accuracy: 0.5361
Epoch 4/10
782/782 [==============================] - 52s 66ms/step - loss: 1.8905 - accuracy: 0.5695
Epoch 5/10
782/782 [==============================] - 59s 76ms/step - loss: 1.8659 - accuracy: 0.5934
Epoch 6/10
782/782 [==============================] - 51s 65ms/step - loss: 1.8434 - accuracy: 0.6162
Epoch 7/10
782/782 [==============================] - 50s 64ms/step - loss: 1.8284 - accuracy: 0.6314
Epoch 8/10
782/782 [==============================] - 50s 64ms/step - loss: 1.8148 - accuracy: 0.6451
Epoch 9/10
782/782 [==============================] - 55s 70ms/step - loss: 1.7945 - accuracy: 0.6665
Epoch 10/10
394/782 [==============>...............] - ETA: 26s - loss: 1.7908 - a

## Testing

In [10]:
# EVALUATE
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

print("ACC: ",test_accuracy)

313/313 [==============================] - 3s 8ms/step - loss: 1.7346 - accuracy: 0.6353
ACC:  0.6352999806404114


In [11]:
# PREDICT
predictions = model.predict(test_images)

print(test_images.shape)
print(predictions.shape)

(10000, 32, 32, 3)
(10000, 10)


## Pretrained Model

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(32,32,3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False # FREEZE WEIGHTS (NON-TRAINABLE)

In [30]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 33, 33, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 16, 16, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 16, 16, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [23]:
custom_head = keras.Sequential([    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax'),
])